# Debug ModelChain - Reprodução do Código Atual

Este notebook reproduz exatamente o código atual do module_service.py para debug.

In [38]:
import pandas as pd
import numpy as np
import requests
from pvlib import pvsystem, modelchain, location, irradiance, solarposition
from pvlib.temperature import TEMPERATURE_MODEL_PARAMETERS
import json

## 1. Buscar Dados PVGIS

In [39]:
# Coordenadas (mesmas do código atual)
lat, lon = -23.7621, -53.3116
tilt, azimuth = 23, 180

print(f"🌍 COORDENADAS: Lat: {lat}, Lon: {lon}")
print(f"📐 ORIENTAÇÃO: Tilt: {tilt}°, Azimute: {azimuth}°")

# Simular dados PVGIS (usar estrutura similar)
url = f"https://re.jrc.ec.europa.eu/api/v5_2/seriescalc?lat={lat}&lon={lon}&startyear=2005&endyear=2020&outputformat=json&usehorizon=1&selectrad=1&angle=0&aspect=0"

print("🔄 Fazendo requisição PVGIS...")
response = requests.get(url, timeout=120)
data = response.json()
hourly_data = data['outputs']['hourly']
print(f"✅ Recebidos {len(hourly_data):,} registros")

🌍 COORDENADAS: Lat: -23.7621, Lon: -53.3116
📐 ORIENTAÇÃO: Tilt: 23°, Azimute: 180°
🔄 Fazendo requisição PVGIS...
✅ Recebidos 140,256 registros


## 2. Processar Dados (Igual ao Código Atual)

In [40]:
def process_pvgis_data(hourly_data):
    """Processa dados PVGIS exatamente como o código atual"""
    processed_records = []
    
    for record in hourly_data:
        try:
            time_str = record['time']
            year = int(time_str[0:4])
            month = int(time_str[4:6])
            day = int(time_str[6:8])
            hour = int(time_str[9:11])
            minute = int(time_str[11:13])
            
            dt = pd.Timestamp(year=year, month=month, day=day, 
                            hour=hour, minute=minute, tz='UTC')
            
            processed_record = {
                'datetime': dt,
                'ghi': float(record['G(i)']),
                'temp_air': float(record['T2m']),
                'wind_speed': float(record['WS10m']),
                'pressure': 101325.0
            }
            processed_records.append(processed_record)
        except (KeyError, ValueError):
            continue
    
    df = pd.DataFrame(processed_records)
    df.set_index('datetime', inplace=True)
    df.index = df.index.tz_convert('America/Sao_Paulo')
    
    # Filtrar anos >= 2005 (igual ao código)
    df = df[df.index.year >= 2005]
    
    return df

df = process_pvgis_data(hourly_data)
print(f"📊 DataFrame processado: {len(df):,} registros")
print(f"   Período: {df.index.min()} até {df.index.max()}")
print(f"   GHI: min={df['ghi'].min():.1f}, max={df['ghi'].max():.1f}, mean={df['ghi'].mean():.1f}")

📊 DataFrame processado: 140,254 registros
   Período: 2005-01-01 00:03:00-02:00 até 2020-12-31 20:03:00-03:00
   GHI: min=0.0, max=1147.0, mean=208.8


## 3. Decomposição GHI → DNI/DHI (DISC vs ERBS)

In [41]:
# Testar DISC (como seu código)
print("=== TESTE COM DISC (como seu código) ===")
df_disc = decompose_ghi(df.copy(), lat, lon, 'disc')

print("\n=== TESTE COM ERBS (como código atual) ===")
df_erbs = decompose_ghi(df.copy(), lat, lon, 'erbs')

# Comparar
print(f"\n📊 COMPARAÇÃO:")
print(f"   DISC - DNI max: {df_disc['dni'].max():.1f} W/m²")
print(f"   ERBS - DNI max: {df_erbs['dni'].max():.1f} W/m²")
print(f"   Diferença: {df_erbs['dni'].max() - df_disc['dni'].max():.1f} W/m²")

# 💾 SALVAR DATAFRAME PARA USAR NO CÓDIGO PYTHON (CSV para compatibilidade)
print(f"\n💾 SALVANDO DATAFRAME PARA CÓDIGO PYTHON:")
df_disc_path = '/Users/rubensfilho/Desktop/Workspace/Soft House/Besspro/bess-pro/debug_df_disc.csv'
df_disc.to_csv(df_disc_path)
print(f"   ✅ Salvo: debug_df_disc.csv")
print(f"   📊 DISC shape: {df_disc.shape}")
print(f"   📊 Colunas: {list(df_disc.columns)}")
print(f"   📊 Index type: {type(df_disc.index)}")

# Mostrar sample dos dados
print(f"\n📋 PRIMEIROS 3 REGISTROS SALVOS:")
print(df_disc.head(3))

=== TESTE COM DISC (como seu código) ===
🔬 DECOMPOSIÇÃO com modelo: DISC
   ✅ Componentes:
      GHI: min=0.0, max=1147.0, mean=208.8
      DNI: min=0.0, max=1010.3, mean=197.9
      DHI: min=0.0, max=542.7, mean=79.6

=== TESTE COM ERBS (como código atual) ===
🔬 DECOMPOSIÇÃO com modelo: ERBS
   ✅ Componentes:
      GHI: min=0.0, max=1147.0, mean=208.8
      DNI: min=0.0, max=1393.8, mean=193.7
      DHI: min=0.0, max=479.9, mean=76.1

📊 COMPARAÇÃO:
   DISC - DNI max: 1010.3 W/m²
   ERBS - DNI max: 1393.8 W/m²
   Diferença: 383.5 W/m²

💾 SALVANDO DATAFRAME PARA CÓDIGO PYTHON:
   ✅ Salvo: debug_df_disc.csv
   📊 DISC shape: (140254, 9)
   📊 Colunas: ['ghi', 'temp_air', 'wind_speed', 'pressure', 'dni', 'dhi', 'solar_zenith', 'solar_azimuth', 'dni_extra']
   📊 Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>

📋 PRIMEIROS 3 REGISTROS SALVOS:
                           ghi  temp_air  wind_speed  pressure  dni  dhi  \
datetime                                                  

## 4. ModelChain Simulation (Código Atual Exato)

In [ ]:
def run_modelchain_simulation(df, lat, lon, tilt, azimuth, model_name="Teste"):
    """ModelChain exatamente igual ao código atual"""
    
    try:
        # Localização
        site = location.Location(latitude=lat, longitude=lon)
        
        # Parâmetros do módulo (iguais ao código atual)
        module_parameters = {
            'alpha_sc': 0.0004,
            'beta_oc': -0.0028,
            'gamma_r': -0.0004,
            'a_ref': 1.8,
            'I_L_ref': 13.91,
            'I_o_ref': 3.712e-12,
            'R_s': 0.348,
            'R_sh_ref': 381.68,
            'cells_in_series': 144,
            'STC': 550,
            'V_oc_ref': 49.7,
            'I_sc_ref': 13.91,
            'V_mp_ref': 41.8,
            'I_mp_ref': 13.16,
            'A0': -3.56, 'A1': -0.075, 'A2': 0.0, 'A3': 0.0, 'A4': 0.0,
            'B0': 0.0, 'B1': 0.0, 'B2': 0.0, 'B3': 0.0, 'B4': 0.0, 'B5': 0.0,
            'DTC': 3.0
        }
        
        # Parâmetros do inversor (iguais ao código atual)
        target_ac = 1 * 550 * 0.85
        inverter_parameters = {
            'Paco': target_ac,
            'Pdco': 1 * 550 * 0.95,
            'Vdco': 360,
            'Pso': 25,
            'C0': -0.000008, 'C1': -0.000120,
            'C2': 0.001400, 'C3': -0.020000,
            'Pnt': 0.02
        }
        
        # Perdas (iguais ao código atual)
        losses_parameters = {
            'soiling': 2.0,
            'shading': 0,
            'mismatch': 2.5,
            'wiring': 2.0
        }
        
        # Sistema
        system = pvsystem.PVSystem(
            surface_tilt=tilt,
            surface_azimuth=azimuth,
            module_parameters=module_parameters,
            inverter_parameters=inverter_parameters,
            modules_per_string=1,
            strings_per_inverter=1,
            temperature_model_parameters=TEMPERATURE_MODEL_PARAMETERS['samp']['open_rack_glass_glass'],
            losses_parameters=losses_parameters
        )
        
        # ModelChain (igual ao código atual - inferência automática)
        mc = modelchain.ModelChain(system, site)
        
        # Debug: Verificar DataFrame completo antes da simulação (igual ao código Python)
        print(f"\n📊 DATAFRAME COMPLETO ANTES DA SIMULAÇÃO - {model_name}:")
        print(f"   Colunas disponíveis: {list(df.columns)}")
        print(f"   Total registros: {len(df):,}")
        print(f"   Período: {df.index.min()} até {df.index.max()}")
        print(f"   GHI: min={df['ghi'].min():.1f}, max={df['ghi'].max():.1f}, mean={df['ghi'].mean():.1f}, std={df['ghi'].std():.1f}")
        print(f"   DNI: min={df['dni'].min():.1f}, max={df['dni'].max():.1f}, mean={df['dni'].mean():.1f}, std={df['dni'].std():.1f}")
        print(f"   DHI: min={df['dhi'].min():.1f}, max={df['dhi'].max():.1f}, mean={df['dhi'].mean():.1f}, std={df['dhi'].std():.1f}")
        print(f"   Temp: min={df['temp_air'].min():.1f}°C, max={df['temp_air'].max():.1f}°C, mean={df['temp_air'].mean():.1f}°C")
        print(f"   Wind: min={df['wind_speed'].min():.1f}, max={df['wind_speed'].max():.1f}, mean={df['wind_speed'].mean():.1f} m/s")
        
        # Verificar se há valores NaN ou inválidos
        nan_counts = df.isnull().sum()
        if nan_counts.any():
            print(f"   ⚠️  Valores NaN encontrados: {nan_counts.to_dict()}")
        
        # Verificar primeiros registros com geração > 0
        sunny_records = df[df['ghi'] > 100]
        if len(sunny_records) > 0:
            print(f"   ☀️  Registros com GHI > 100 W/m²: {len(sunny_records):,}")
            print(f"   ☀️  Primeiro registro ensolarado:")
            first_sunny = sunny_records.iloc[0]
            print(f"       Data: {first_sunny.name}")
            print(f"       GHI: {first_sunny['ghi']:.1f}, DNI: {first_sunny['dni']:.1f}, DHI: {first_sunny['dhi']:.1f}")
            print(f"       Temp: {first_sunny['temp_air']:.1f}°C, Wind: {first_sunny['wind_speed']:.1f} m/s")
        else:
            print(f"   ❌ PROBLEMA: Nenhum registro com GHI > 100 W/m²")

        # Executar
        mc.run_model(df)
        print(mc.results)
    
        
        # Debug: Verificar se mc.results foi populado (igual ao código Python)
        print(f"\n🔍 MC RESULTS APÓS SIMULAÇÃO - {model_name}:")
        print(f"   AC disponível: {mc.results.ac is not None}")
        if mc.results.ac is not None:
            print(f"   AC length: {len(mc.results.ac)}")
            print(f"   AC raw: min={mc.results.ac.min():.3f}, max={mc.results.ac.max():.3f}, mean={mc.results.ac.mean():.3f}")
        
        if hasattr(mc.results, 'dc') and mc.results.dc is not None:
            print(f"   DC disponível com {len(mc.results.dc)} registros")
        
        if hasattr(mc.results, 'poa_global') and mc.results.poa_global is not None:
            print(f"   POA disponível: mean={mc.results.poa_global.mean():.1f} W/m²")
        
        if mc.results.ac is None or len(mc.results.ac) == 0:
            return None
        
        # ANÁLISE SIMPLIFICADA
        print(f"\n📊 ANÁLISE FINAL - {model_name}:")
        print(f"   Total AC (soma tudo): {mc.results.ac.sum():.2f} W")
        print(f"   Média AC: {mc.results.ac.mean():.6f} W")
        
        # Média AC agrupado por dia
        daily_mean_ac = mc.results.ac.groupby(mc.results.ac.index.date).mean()
        print(f"   Média AC por dia (primeiros 10 dias):")
        for date, mean_power in daily_mean_ac.head(10).items():
            print(f"      {date}: {mean_power:.6f} W")
            
        # NOVA ANÁLISE: Média de todas as médias diárias
        print(f"\n   📈 ANÁLISE COMPLETA DE MÉDIAS DIÁRIAS:")
        print(f"      Total de dias: {len(daily_mean_ac)}")
        print(f"      Soma de todas médias diárias: {daily_mean_ac.sum():.2f} W")
        print(f"      Média das médias diárias: {daily_mean_ac.mean():.6f} W")
        print(f"      Min média diária: {daily_mean_ac.min():.6f} W")
        print(f"      Max média diária: {daily_mean_ac.max():.6f} W")
        
        # Calcular energia anual por ano
        annual_energy_by_year = mc.results.ac.groupby(mc.results.ac.index.year).sum() / 1000
        
        result = {
            'mean': annual_energy_by_year.mean(),
            'std': annual_energy_by_year.std(),
            'by_year': annual_energy_by_year.to_dict(),
            'ac_results': mc.results.ac,
            'daily_mean_ac': daily_mean_ac
        }
        
        return result
        
    except Exception as e:
        print(f"   ❌ Erro na simulação {model_name}: {e}")
        return None

## 5. Executar Testes Comparativos

In [9]:
# Teste 1: DISC (como seu código)
result_disc = run_modelchain_simulation(df_disc, lat, lon, tilt, azimuth, "DISC")

# Teste 2: ERBS (como código atual)
result_erbs = run_modelchain_simulation(df_erbs, lat, lon, tilt, azimuth, "ERBS")

# Comparação final
print(f"\n" + "="*60)
print(f"🏁 RESULTADO FINAL:")

if result_disc:
    print(f"   DISC (seu código):    {result_disc['mean']:.3f} kWh/ano")
else:
    print(f"   DISC (seu código):    FALHOU")

if result_erbs:
    print(f"   ERBS (código atual):  {result_erbs['mean']:.3f} kWh/ano")
else:
    print(f"   ERBS (código atual):  FALHOU")

print(f"="*60)


⚡ SIMULAÇÃO MODELCHAIN - DISC
   📋 Parâmetros inversor: Paco=468W
   🔄 Executando ModelChain...
      Total registros: 140,254
      Registros com GHI > 50: 58,430
   ✅ ModelChain executado com sucesso
      AC output: min=-0.020000, max=244.664206, mean=29.507498

   📊 TODOS OS VALORES AC (primeiros 30):
datetime
2005-01-01 00:03:00-02:00     -0.020000
2005-01-01 01:03:00-02:00     -0.020000
2005-01-01 02:03:00-02:00     -0.020000
2005-01-01 03:03:00-02:00     -0.020000
2005-01-01 04:03:00-02:00     -0.020000
2005-01-01 05:03:00-02:00     -0.020000
2005-01-01 06:03:00-02:00     -0.020000
2005-01-01 07:03:00-02:00     -0.020000
2005-01-01 08:03:00-02:00     37.005142
2005-01-01 09:03:00-02:00     84.613456
2005-01-01 10:03:00-02:00    100.448838
2005-01-01 11:03:00-02:00    112.772709
2005-01-01 12:03:00-02:00    129.247434
2005-01-01 13:03:00-02:00    123.675267
2005-01-01 14:03:00-02:00    113.985840
2005-01-01 15:03:00-02:00     70.399997
2005-01-01 16:03:00-02:00     46.322788
200

## 6. Debug Detalhado (Se Necessário)

In [7]:
# Inspecionar dados detalhados se algum teste falhou
print("🔍 DEBUG DETALHADO:")

if result_erbs is None:
    print("\n⚠️  ERBS falhou - investigando...")
    
    # Verificar se há valores inválidos
    invalid_dni = (df_erbs['dni'] > 1200).sum()
    invalid_dhi = (df_erbs['dhi'] < 0).sum()
    invalid_ghi = (df_erbs['ghi'] < 0).sum()
    
    print(f"   DNI > 1200 W/m²: {invalid_dni:,} registros")
    print(f"   DHI < 0 W/m²: {invalid_dhi:,} registros")
    print(f"   GHI < 0 W/m²: {invalid_ghi:,} registros")
    
    if invalid_dni > 0:
        print(f"\n   🚨 PROBLEMA: DNI com valores fisicamente impossíveis!")
        print(f"      DNI máximo: {df_erbs['dni'].max():.1f} W/m² (limite ~1200)")
        print(f"      Sugestão: Clippar DNI ou usar modelo DISC")

print(f"\n📋 PRIMEIROS VALORES ERBS:")
print(df_erbs[df_erbs['ghi'] > 0][['ghi', 'dni', 'dhi']].head(10))

print(f"\n📋 VALORES MÁXIMOS ERBS:")
max_idx = df_erbs['dni'].idxmax()
print(f"DNI máximo em {max_idx}: {df_erbs.loc[max_idx, 'dni']:.1f} W/m²")
print(f"GHI correspondente: {df_erbs.loc[max_idx, 'ghi']:.1f} W/m²")
print(f"DHI correspondente: {df_erbs.loc[max_idx, 'dhi']:.1f} W/m²")

🔍 DEBUG DETALHADO:

📋 PRIMEIROS VALORES ERBS:
                              ghi         dni         dhi
datetime                                                 
2005-01-01 08:03:00-02:00   120.0   38.225476  110.324829
2005-01-01 09:03:00-02:00   327.0  232.860368  218.385771
2005-01-01 10:03:00-02:00   580.0  537.008942  226.262663
2005-01-01 11:03:00-02:00   803.0  735.794767  201.961154
2005-01-01 12:03:00-02:00   955.0  813.582342  198.306200
2005-01-01 13:03:00-02:00  1053.0  869.834782  191.288625
2005-01-01 14:03:00-02:00  1070.0  887.678402  187.222698
2005-01-01 15:03:00-02:00  1051.0  932.735370  173.048772
2005-01-01 16:03:00-02:00   945.0  945.407361  155.925000
2005-01-01 17:03:00-02:00   651.0  682.019814  185.956065

📋 VALORES MÁXIMOS ERBS:
DNI máximo em 2005-04-01 18:03:00-03:00: 1393.8 W/m²
GHI correspondente: 168.0 W/m²
DHI correspondente: 27.7 W/m²
